### V2 OMOP Pediatric Referral

In [2]:
##Setting up Google sdk environment
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/wui/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

##Setting up BQ API
from google.cloud import bigquery
client = bigquery.Client()
project_id = 'som-rit-phi-starr-prod'
dataset_id = 'starr_omop_cdm5_deid_latest'

/share/sw/open/anaconda/3/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [6]:
# generate a table of all outpatient visits with pediatric patients (less than 18 years of age)
# visit_concept_id 9202 = outpatient visit 

sql = """
SELECT 
    v.person_id,
    v.visit_occurrence_id,
    p.birth_DATETIME,
    v.visit_start_DATETIME,
    v.provider_id,
    v.care_site_id,
    v.visit_source_value,
    v.preceding_visit_occurrence_id
FROM 
    `{project_id}.{dataset_id}.visit_occurrence` v
LEFT JOIN
    `{project_id}.{dataset_id}.person` p ON v.person_id = p.person_id
WHERE 
    visit_concept_id IN (0,9202) AND 
    (DATETIME_DIFF(v.visit_start_DATETIME, p.birth_DATETIME, DAY) BETWEEN 0 AND 6575) AND
    DATETIME_DIFF(v.visit_end_DATETIME, v.visit_start_DATETIME, DAY) = 0 AND
    EXTRACT(YEAR FROM v.visit_start_DATETIME) >= 2015 AND
    EXTRACT(YEAR FROM v.visit_start_DATETIME) < 2020
""".format_map({'project_id': project_id,'dataset_id': dataset_id})
table_id = "som-nero-phi-jonc101.wui_omop_peds.V2_Outpt_Visit_2015_2019"
job_config = bigquery.QueryJobConfig(destination=table_id)
client.delete_table(table_id, not_found_ok = True)
query_job = client.query(sql, job_config=job_config)  
query_job.result() 
print("Query results loaded to the table {}".format(table_id))


Query results loaded to the table som-nero-phi-jonc101.wui_omop_peds.V2_Outpt_Visit_2015_2019


In [8]:
# how many outpatient pediatric clinic visits have a referral order to endocrine in 2015-2019
sql = """
SELECT
    v.person_id,
    v.visit_occurrence_id,
    pr.procedure_DATETIME as referral_DATETIME,
    pr.procedure_source_value AS referral
FROM 
    `{project_id}.{dataset_id}.procedure_occurrence` pr
INNER JOIN
    `som-nero-phi-jonc101.wui_omop_peds.V2_Outpt_Visit_2015_2019` v 
ON 
    pr.person_id = v.person_id AND pr.visit_occurrence_id = v.visit_occurrence_id
WHERE 
    REGEXP_CONTAINS(procedure_source_value, r'(?i)referral') AND
    REGEXP_CONTAINS(procedure_source_value, r'(?i)endo')
""".format_map({'project_id': project_id,'dataset_id': dataset_id})
table_id = "som-nero-phi-jonc101.wui_omop_peds.V2_Endo_Referral_2015_2019"
job_config = bigquery.QueryJobConfig(destination=table_id)
client.delete_table(table_id, not_found_ok = True)
query_job = client.query(sql, job_config=job_config)  
query_job.result() 
print("Query results loaded to the table {}".format(table_id))



Query results loaded to the table som-nero-phi-jonc101.wui_omop_peds.V2_Endo_Referral_2015_2019


In [9]:
# All Endocrine visits 2015 - 2019
# specialty_dep_c: 81 - Diabetes, 7 - Endocrinology, 106 - Pediatric Endocrinology 
sql = """
SELECT 
    v.person_id,
    v.visit_occurrence_id,
    v.visit_start_DATETIME,
    c.care_site_id,
    c.care_site_name,
    'Endocrine' AS specialty
FROM 
    `som-nero-phi-jonc101.wui_omop_peds.V2_Outpt_Visit_2015_2019` v
LEFT JOIN
    `{project_id}.{dataset_id}.care_site` c ON v.care_site_id = c.care_site_id
LEFT JOIN 
    `som-nero-phi-jonc101.starr_datalake2018.dep_map` m ON CAST(c.care_site_source_value AS NUMERIC) = m.department_id
WHERE 
    m.specialty_dep_c in ('7','81','106')
""".format_map({'project_id': project_id,'dataset_id': dataset_id})

table_id = "som-nero-phi-jonc101.wui_omop_peds.V2_Endo_Visit_2015_2019"
job_config = bigquery.QueryJobConfig(destination=table_id)
client.delete_table(table_id, not_found_ok = True)
query_job = client.query(sql, job_config=job_config)  
query_job.result() 
print("Query results loaded to the table {}".format(table_id))

Query results loaded to the table som-nero-phi-jonc101.wui_omop_peds.V2_Endo_Visit_2015_2019


In [10]:
# All Primary Care visits 2015 - 2019
# 72 adolescent medicine
# 32 Pediatrics 
sql = """
SELECT 
    v.person_id,
    v.visit_occurrence_id,
    v.visit_start_DATETIME,
    c.care_site_id,
    c.care_site_name,
    'Primary Care' AS specialty
FROM 
    `som-nero-phi-jonc101.wui_omop_peds.V2_Outpt_Visit_2015_2019` v
LEFT JOIN
    `{project_id}.{dataset_id}.care_site` c ON v.care_site_id = c.care_site_id
LEFT JOIN 
    `som-nero-phi-jonc101.starr_datalake2018.dep_map` m ON CAST(c.care_site_source_value AS NUMERIC) = m.department_id
WHERE 
    m.specialty_dep_c in ('32','72')
""".format_map({'project_id': project_id,'dataset_id': dataset_id})

table_id = "som-nero-phi-jonc101.wui_omop_peds.V2_PrimaryCare_Visit_2015_2019"
job_config = bigquery.QueryJobConfig(destination=table_id)
client.delete_table(table_id, not_found_ok = True)
query_job = client.query(sql, job_config=job_config)  
query_job.result() 
print("Query results loaded to the table {}".format(table_id))

Query results loaded to the table som-nero-phi-jonc101.wui_omop_peds.V2_PrimaryCare_Visit_2015_2019


In [11]:
# Joining primary care visits and endocrine visits via endocrine referral 
sql = """
SELECT 
   PrimaryCare.person_id,
   PrimaryCare.visit_occurrence_id as PrimaryCare_visit_id,
   PrimaryCare.care_site_name as PrimaryCare_care_site,
   PrimaryCare.visit_start_DATETIME as PrimaryCare_DATETIME,
FROM 
    `som-nero-phi-jonc101.wui_omop_peds.V2_PrimaryCare_Visit_2015_2019` PrimaryCare
INNER JOIN 
    `som-nero-phi-jonc101.wui_omop_peds.V2_Endo_Referral_2015_2019` EndoRefer 
            ON 
                PrimaryCare.person_id = EndoRefer.person_id AND 
                PrimaryCare.visit_occurrence_id = EndoRefer.visit_occurrence_id    
                            
ORDER BY 
    PrimaryCare.person_id, PrimaryCare.visit_start_DATETIME

""".format_map({'project_id': project_id,'dataset_id': dataset_id})


table_id = "som-nero-phi-jonc101.wui_omop_peds.V2_PrimaryCare_EndoRefer_2015_2019"
job_config = bigquery.QueryJobConfig(destination=table_id)
client.delete_table(table_id, not_found_ok = True)
query_job = client.query(sql, job_config=job_config)  
query_job.result() 
print("Query results loaded to the table {}".format(table_id))


#    PrimaryCare_Note.note_title as PrimaryCare_notetitle,
#    PrimaryCare_Note.note_type_concept_id,
#    PrimaryCare_Note.note_class_concept_id,
#    Endocrine_Note.note_title as Endo_notetitle

# LEFT JOIN 
#     `{project_id}.{dataset_id}.note` PrimaryCare_Note 
#             ON 
#                 PrimaryCare.person_id = PrimaryCare_Note.person_id AND
#                 PrimaryCare.visit_occurrence_id = PrimaryCare_Note.visit_occurrence_id
# LEFT JOIN 
#     `{project_id}.{dataset_id}.note` Endocrine_Note
#             ON 
#                 PrimaryCare.person_id = Endocrine_Note.person_id AND


Query results loaded to the table som-nero-phi-jonc101.wui_omop_peds.V2_PrimaryCare_EndoRefer_2015_2019


In [12]:
# Joining primary care visits with endocrine referral to actual Endocrine visit  
sql = """
SELECT 
   EndoRefer.person_id,
   EndoRefer.PrimaryCare_visit_id,
   EndoRefer.PrimaryCare_care_site,
   EndoRefer.PrimaryCare_DATETIME,
   Endocrine.visit_occurrence_id as Endo_visit_id,
   Endocrine.care_site_name as Endo_care_site,
   Endocrine.visit_start_DATETIME as Endo_DATETIME,
FROM 
    `som-nero-phi-jonc101.wui_omop_peds.V2_PrimaryCare_EndoRefer_2015_2019` EndoRefer
INNER JOIN
    `som-nero-phi-jonc101.wui_omop_peds.V2_Endo_Visit_2015_2019` Endocrine
            ON 
                EndoRefer.person_id = Endocrine.person_id AND
                DATETIME_DIFF(Endocrine.visit_start_DATETIME, EndoRefer.PrimaryCare_DATETIME, MONTH) BETWEEN 0 AND 11
                            
ORDER BY 
    EndoRefer.person_id, EndoRefer.PrimaryCare_DATETIME

""".format_map({'project_id': project_id,'dataset_id': dataset_id})


table_id = "som-nero-phi-jonc101.wui_omop_peds.V2_EndoRefer_Endocrine_2015_2019"
job_config = bigquery.QueryJobConfig(destination=table_id)
client.delete_table(table_id, not_found_ok = True)
query_job = client.query(sql, job_config=job_config)  
query_job.result() 
print("Query results loaded to the table {}".format(table_id))



Query results loaded to the table som-nero-phi-jonc101.wui_omop_peds.V2_EndoRefer_Endocrine_2015_2019


In [13]:
# OMOP-CDM does not distinguish visit types (office visit, telephone encounters, etc)
# Define Office Visit as a visit with a progress note (note_class_concept_id = 3000735)

sql = """
WITH 
    Office_Visit AS 
        (SELECT visit_occurrence_id FROM `{project_id}.{dataset_id}.note` 
            WHERE note_class_concept_id = 3000735) 
SELECT 
    *,
    ROW_NUMBER() OVER (PARTITION BY PrimaryCare_visit_id ORDER BY Endo_DATETIME) AS Endo_visit_rank
FROM 
   `som-nero-phi-jonc101.wui_omop_peds.V2_PrimaryCare_Endocrine_2015_2019` 
WHERE 
    PrimaryCare_visit_id IN (SELECT * FROM Office_Visit) AND
    Endo_visit_id IN (SELECT * FROM Office_Visit) 
ORDER BY 
    person_id, PrimaryCare_DATETIME
""".format_map({'project_id': project_id,'dataset_id': dataset_id})
table_id = "som-nero-phi-jonc101.wui_omop_peds.V2_Office_PrimaryCare_Endocrine_2015_2019"
job_config = bigquery.QueryJobConfig(destination=table_id)
client.delete_table(table_id, not_found_ok = True)
query_job = client.query(sql, job_config=job_config)  
query_job.result() 
print("Query results loaded to the table {}".format(table_id))

Query results loaded to the table som-nero-phi-jonc101.wui_omop_peds.V2_Office_PrimaryCare_Endocrine_2015_2019


In [14]:
# OMOP-CDM does not distinguish visit types (office visit, telephone encounters, etc)
# Define Office Visit as a visit with a progress note (note_class_concept_id = 3000735)
# Different from the above cell in that we are not restricting primary care visit with a progress note
# it turns out there are a good number of endo referrals were made during a telephone encounter 
# lab results are back

sql = """
WITH 
    Office_Visit AS 
        (SELECT visit_occurrence_id FROM `{project_id}.{dataset_id}.note` 
            WHERE note_class_concept_id = 3000735) 
SELECT 
    *,
    ROW_NUMBER() OVER (PARTITION BY PrimaryCare_visit_id ORDER BY Endo_DATETIME) AS Endo_visit_rank
FROM 
   `som-nero-phi-jonc101.wui_omop_peds.V2_PrimaryCare_Endocrine_2015_2019` 
WHERE 
    Endo_visit_id IN (SELECT * FROM Office_Visit) 
ORDER BY 
    person_id, PrimaryCare_DATETIME
""".format_map({'project_id': project_id,'dataset_id': dataset_id})
table_id = "som-nero-phi-jonc101.wui_omop_peds.V2_PrimaryCare_Office_Endocrine_2015_2019"
job_config = bigquery.QueryJobConfig(destination=table_id)
client.delete_table(table_id, not_found_ok = True)
query_job = client.query(sql, job_config=job_config)  
query_job.result() 
print("Query results loaded to the table {}".format(table_id))

Query results loaded to the table som-nero-phi-jonc101.wui_omop_peds.V2_PrimaryCare_Office_Endocrine_2015_2019


In [ ]:
# query to check the difference between the above two cells 
""""
WITH extraPrimaryCare AS(
  SELECT *  FROM `som-nero-phi-jonc101.wui_omop_peds.V2_PrimaryCare_Office_Endocrine_2015_2019` 
    WHERE PrimaryCare_visit_id NOT IN 
    (SELECT PrimaryCare_visit_id FROM `som-nero-phi-jonc101.wui_omop_peds.V2_Office_PrimaryCare_Endocrine_2015_2019`)
    AND Endo_visit_rank = 1)
    
SELECT person_id, visit_occurrence_id, note_DATETIME, note_title, note_text  FROM `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.note` WHERE visit_occurrence_id IN
     (SELECT PrimaryCare_visit_id FROM extraPrimaryCare) 
ORDER BY person_id, visit_occurrence_id, note_class_concept_id""""


In [ ]:
# query to check the primary care visit with Endo Referral that has visit type concept id = 0
"""WITH visitTypeNoMatch AS 
(
 SELECT * FROM
  `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.visit_occurrence` 
WHERE 
  visit_occurrence_id IN 
      (SELECT DISTINCT(PrimaryCare_visit_id) FROM `som-nero-phi-jonc101.wui_omop_peds.V2_PrimaryCare_EndoRefer_2015_2019`)
    AND
  visit_concept_id = 0
)

SELECT 
  visit_start_DATETIME, visit_type_concept_id, c1.concept_name, v.care_site_id, c.care_site_name   
FROM 
  visitTypeNoMatch v
LEFT JOIN 
  `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.care_site` c ON v.care_site_id = c.care_site_id 
LEFT JOIN
  `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.concept` c1 ON v.visit_type_concept_id = c1.concept_id """

In [ ]:
# another way to get endocrine visit using regular expression over care site name 

WITH outpatient AS
     (SELECT DISTINCT(care_site_id) FROM `som-nero-phi-jonc101.wui_omop_peds.V2_Outpt_Visit_2015_2019`)

SELECT * FROM `som-nero-phi-jonc101.wui_omop_peds.V2_Outpt_Visit_2015_2019` v
WHERE v.care_site_id IN
(
SELECT o.care_site_id
FROM outpatient o 
LEFT JOIN 
   `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.care_site` c 
ON o.care_site_id = c.care_site_id
WHERE REGEXP_CONTAINS(care_site_name, r'(?i)endo|diab') AND NOT REGEXP_CONTAINS(care_site_name, r'(?i)repro|endoscopy|oncology') )